In [50]:
import requests
import time
import datetime as dt

In [55]:
def query_pushshift(subreddit, kind='submission', skip=1, times=2, 
                    subfield = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self'],
                    comfields = ['body', 'score', 'created_utc']):

    stem = "https://api.pushshift.io/reddit/search/{}/?subreddit={}&size=1000".format(kind, subreddit)
    mylist = []
    
    for x in range(1, times + 1):
        
        URL = "{}&after={}d".format(stem, skip * x)
        print(URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        mylist.append(df)
        time.sleep(2)
        
    full = pd.concat(mylist, sort=False)
    
    if kind == "submission":
        
        full = full[subfield]
        
        full = full.drop_duplicates()
        
        full = full.loc[full['is_self'] == True]
        
    def get_date(created):
        return dt.date.fromtimestamp(created)
    
    _timestamp = full["created_utc"].apply(get_date)
    
    full['timestamp'] = _timestamp

    print(full.shape)
    
    return full 

In [58]:
math_for_predictions = query_pushshift('math')

https://api.pushshift.io/reddit/search/submission/?subreddit=math&size=1000&after=1d
https://api.pushshift.io/reddit/search/submission/?subreddit=math&size=1000&after=2d
(115, 9)


In [61]:
math_for_predictions.drop(columns=['created_utc', 'author', 'num_comments', 'score', 'is_self', 'timestamp'], inplace=True)

In [63]:
math_for_predictions['post'] = math_for_predictions['title'] +' '+ math_for_predictions['selftext']

In [66]:
math_for_predictions.drop(columns=['title', 'selftext', 'subreddit'], inplace=True)

In [67]:
math_for_predictions.head()

,post
1,Who’s in Full Burn Out Mode? The burn out is r...
2,Tweeted [removed]
3,Any ideas where to begin my educational journe...
5,need help with theorems [removed]
6,Real world applications of calculus ? Examples...


In [68]:
math_for_predictions.to_csv('math_for_predictions.csv', index=False)